In [1]:
import gdown
import pandas as pd
import os
import csv
import re

In [2]:
def download_csv():
    sheet_url = "https://docs.google.com/spreadsheets/d/1vYD9nyEoI2CEWu2UY2xpedC_4mX3TUVD-XLCjOLphiA/export?format=csv"

    # Load the CSV into a DataFrame
    df = pd.read_csv(sheet_url)
    df.to_csv("form.csv")
    # print(df)

In [3]:
download_csv()

In [4]:
original_csv = "form.csv"
processed_csv = "processed.csv"
clothes_folder = "clothes_data"
possible_columns = [
    "服裝部位 Clothing Part",
    "上半身 衣服種類 Top Clothing Type",
    "下半身衣服種類 Bottom Clothing Type",
]

In [5]:
df = pd.read_csv(original_csv)
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df2 = df.drop("時間戳記", axis=1)
for col in possible_columns:
    df2 = df2.drop(col, axis=1)

In [6]:
# get all feature
all_features = []
for index, row in df.iterrows():
    for col in possible_columns:
        str = df.loc[index, col]
        try:
            str = list(str.split(","))
            for s in str:
                s = s.rstrip()
                s = s.lstrip()
                if (
                    s == "針織衣 Sweater"
                    or s == "毛衣 Sweater"
                    or s == "針/織衣 Sweater"
                    or s == "毛衣、針/織衣 Sweater"
                ):
                    s = "毛衣、針織衣 Sweater"
                if s == "西裝褲 dress pants":
                    str.append("褲子 pants")
                    s = "西裝褲、裙 dress pants、skirt"
                all_features.append(s)
        except:
            pass
all_features = set(all_features)
for f in all_features:
    print(f)
print(len(all_features))

毛衣、針織衣 Sweater
休閒褲/裙 casual pants/skirt
上半身 Top
Shirt
運動褲/裙 sweat pants / sports skirt
外套 Jacket
裙子 skirt
網紗 Mesh
連帽衣 Hoodie
運動材質 Sport Tops
吊帶褲/裙 overalls/jumper skirt
百褶裙 pleated skirt
西裝 Suit
羽絨衣 Down coat / jacket
工裝褲/裙 cargo pants/skirt
褲子 pants
牛仔材質 Denim
毛衣、針/織衣 Sweater
背心 Vest
牛仔褲/裙 jeans / demi skirt
洋裝 Dress
緊身褲 leggings
T-Shirt
西裝褲、裙 dress pants、skirt
下半身 Bottom
25


In [7]:
for index, row in df.iterrows():
    for fea in all_features:
        has = 0
        for col in possible_columns:
            str = df.loc[index, col]
            try:
                str = list(str.split(","))
                for s in str:
                    s = s.rstrip()
                    s = s.lstrip()
                    if (
                        s == "針織衣 Sweater"
                        or s == "毛衣 Sweater"
                        or s == "針/織衣 Sweater"
                        or s == "毛衣、針/織衣 Sweater"
                    ):
                        s = "毛衣、針織衣 Sweater"
                    if s == "西裝褲 dress pants":
                        str.append("褲子 pants")
                        s = "西裝褲、裙 dress pants、skirt"
                    if s == fea:
                        has = 1
            except:
                pass
        df2.loc[index, fea] = has


print(df2)

                                             照片 Picture 品牌 Brand  \
0     https://drive.google.com/open?id=1Ki8gALrL6WEy...      H&M   
1     https://drive.google.com/open?id=1rsB7XhHjAww6...      H&M   
2     https://drive.google.com/open?id=1BEUPTgjmZ0uZ...      H&M   
3     https://drive.google.com/open?id=12MW0xvZVVNVo...      H&M   
4     https://drive.google.com/open?id=1rj0PNtlTPp5O...      H&M   
...                                                 ...      ...   
2208  https://drive.google.com/open?id=1eCfhQbeZqXOX...    PAZZO   
2209  https://drive.google.com/open?id=1NMx8fObRHc_U...    PAZZO   
2210  https://drive.google.com/open?id=1OY30Q91RIN5p...    PAZZO   
2211  https://drive.google.com/open?id=1mfpqzS7FzFEM...    PAZZO   
2212  https://drive.google.com/open?id=1-3g0FKDFwMzd...    PAZZO   

     衣服長度 Clothes Length  價格 Price  毛衣、針織衣 Sweater  休閒褲/裙 casual pants/skirt  \
0                短 short       499             0.0                       0.0   
1                 長 lon

In [8]:
for index, row in df2.iterrows():
    image_url = row["照片 Picture"]  # 從表單中的照片 URL 讀取

    if re.search(r"open\?id=", image_url):
        output = f"image_{index:05}.jpg"
        df2.loc[index, "照片 Picture"] = output
df2.to_csv(processed_csv)

# check image downloaded or not

In [9]:
def download_image(image_url, path):
    file_id = image_url.split("open?id=")[-1]
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    gdown.download(url, path, quiet=False)

In [10]:
for index, row in df2.iterrows():
    output = f"image_{index:05}.jpg"
    path = os.path.join(clothes_folder, output)
    if not os.path.exists(path):
        image_url = df.loc[index, "照片 Picture"]
        print("download ", index, " from ", image_url)
        download_image(image_url, path)

download  825  from  https://drive.google.com/open?id=1s9Om0HGafnTY_2yklDNxB-QbpvP8BHB5


Downloading...
From: https://drive.google.com/uc?export=download&id=1s9Om0HGafnTY_2yklDNxB-QbpvP8BHB5
To: c:\Users\doo12\Desktop\11301\ML\finalproject\ClothesPredicting\clothes_data\image_00825.jpg
100%|██████████| 115k/115k [00:00<00:00, 2.34MB/s]


download  826  from  https://drive.google.com/open?id=1EXFr7rTfju9yrtDArJdUSt4LYWaP8FQ_


Downloading...
From: https://drive.google.com/uc?export=download&id=1EXFr7rTfju9yrtDArJdUSt4LYWaP8FQ_
To: c:\Users\doo12\Desktop\11301\ML\finalproject\ClothesPredicting\clothes_data\image_00826.jpg
100%|██████████| 103k/103k [00:00<00:00, 1.85MB/s]


download  827  from  https://drive.google.com/open?id=1e5e-V1GPvKayY6hYTXpBH3gqaDzoyPLJ


...

Downloading...
From: https://drive.google.com/uc?export=download&id=1OY30Q91RIN5p6Pg7aie5oUVsoxBYD534
To: c:\Users\doo12\Desktop\11301\ML\finalproject\ClothesPredicting\clothes_data\image_02210.jpg
100%|██████████| 26.7k/26.7k [00:00<00:00, 25.4MB/s]


download  2211  from  https://drive.google.com/open?id=1mfpqzS7FzFEMY8TZSNRTNO0U7tGr0HDa


Downloading...
From: https://drive.google.com/uc?export=download&id=1mfpqzS7FzFEMY8TZSNRTNO0U7tGr0HDa
To: c:\Users\doo12\Desktop\11301\ML\finalproject\ClothesPredicting\clothes_data\image_02211.jpg
100%|██████████| 27.0k/27.0k [00:00<00:00, 9.00MB/s]


download  2212  from  https://drive.google.com/open?id=1-3g0FKDFwMzd5EHfko2SX9fV_NHESsyT


Downloading...
From: https://drive.google.com/uc?export=download&id=1-3g0FKDFwMzd5EHfko2SX9fV_NHESsyT
To: c:\Users\doo12\Desktop\11301\ML\finalproject\ClothesPredicting\clothes_data\image_02212.jpg
100%|██████████| 40.8k/40.8k [00:00<00:00, 1.63MB/s]
